# OpenAI Agent 

OpenAI 플랫폼을 사용하여 사용자를 대신해 행동(예: 컴퓨터 제어 등)을 수행할 수 있는 **에이전트(Agent)** 를 구축할 수 있습니다.  
Python용 **Agent SDK**를 사용하면 백엔드에서 이러한 에이전트의 **오케스트레이션(조율) 로직**을 만들 수 있습니다.

**OpenAI Agents SDK**는 **에이전트 기반 AI 애플리케이션**을 개발할 수 있게 해주는 도구입니다.  

**핵심 구성 요소**  

- **에이전트 (Agents)**:  
  LLM에 **지시문(instructions)** 과 **도구(tools)** 를 장착한 실행 단위  
- **핸드오프 (Handoffs)**:  
  특정 작업을 다른 에이전트에게 **위임**할 수 있도록 해줌  
- **가드레일 (Guardrails)**:  
  에이전트에게 전달되는 입력을 **검증**하는 장치

### 주요 기능 요약:

- **에이전트 루프 (Agent Loop)**:  
  도구 실행 → 결과 전달 → LLM 호출 → 반복 실행 → 완료까지 자동 처리

- **핸드오프 (Handoffs)**:  
  여러 에이전트 간의 **협업과 위임**을 유연하게 처리 가능

- **가드레일 (Guardrails)**:  
  에이전트 입력을 **사전 검사/검증**하여, 조건을 만족하지 않으면 **조기 종료 가능**

- **함수 기반 도구 (Function Tools)**:  
  Python 함수 하나를 **자동으로 에이전트 도구로 변환**,  
  **Pydantic 기반 스키마 자동 생성** 및 검증 포함

- **추적(Tracing)**:  
  워크플로우를 **시각화/디버깅/모니터링** 가능하며,  
  OpenAI의 평가/파인튜닝/디스틸레이션 툴과 통합 가능

In [ ]:
# pip install openai-agents

### Hello World 예제

| 메서드                        | 호출 방식                   | 특징                                  |
| -------------------------- | ----------------------- | ----------------------------------- |
| `Runner.run(...)`          | `await Runner.run(...)` | 비동기적, 에이전트 루프 자동 실행, 도구 & 핸드오프 지원   |
| `Runner.run_sync(...)`     | `Runner.run_sync(...)`  | 동기 실행으로 첫 번째 메서드 래핑, 스크립트/테스트 환경 적합 |
| `Runner.run_streamed(...)` | 비동기 + 스트리밍              | 중간 응답을 이벤트로 실시간 전송 가능               |


In [ ]:
# Agent(에이전트 정의)와 Runner(실행 관리자) 불러오기
# 비동기적으로 에이전트를 실행하여 사용자 요청에 대한 응답을 받음
# 요청: "재귀적 프로그래밍에 대한 짧은 시를 3줄 이내로 써주세요."
# 최종 응답 결과를 출력

### Simple Handoff Example

언어에 따라 적절한 에이전트에 작업을 위임(handoff)

In [ ]:
# 한국어 에이전트 생성: 한국어만 사용 가능
# 영어 에이전트 생성: 영어만 사용 가능
# 분류 역할의 핸드오프 에이전트 생성
# 입력된 문장의 언어를 판별하여 적절한 에이전트(한국어 or 영어)에게 전달
# Agent orchenstration 실행

------------------------
## 도구 (Tools)

**도구(Tools)** 는 에이전트가 **행동을 수행할 수 있도록 해주는 기능**입니다.  
예를 들어, **데이터를 가져오거나**, **코드를 실행하고**, **외부 API를 호출하거나**, 심지어 **컴퓨터를 사용하는 작업**까지도 가능합니다.

Agent SDK에서는 도구를 다음 **세 가지 유형**으로 구분합니다:  

### 1. **Hosted tools (호스팅 도구)**  
이 도구들은 **LLM 서버 내에서**, AI 모델과 함께 실행됩니다.  
OpenAI는 다음과 같은 호스팅 도구를 제공합니다:
- 검색 기반 정보 검색 (Retrieval)
- 웹 검색 (Web search)
- 컴퓨터 사용 (Computer use)

### 2. **Function calling (함수 호출 도구)**  
이 방식은 **Python 함수**를 도구로 등록하여 사용할 수 있게 해줍니다. 즉, 여러분이 직접 정의한 함수도 LLM이 도구처럼 호출할 수 있습니다.  

**Agents SDK**에서는 **모든 Python 함수를 도구(tool)** 로 사용할 수 있습니다. SDK는 이러한 도구를 자동으로 설정해줍니다:

1. **도구 이름**은 Python 함수의 이름에서 자동으로 생성됩니다.  

2. **도구 설명(description)** 은 함수의 **docstring**에서 가져옵니다.  

3. 함수 입력값의 **스키마(schema)** 는 함수의 **인자(arguments)** 로부터 자동 생성됩니다.

4. 각 입력값에 대한 **설명**도 함수의 docstring에서 추출됩니다.  

### 3. **Agents as tools (에이전트를 도구처럼 사용)**  
이 방식은 **하나의 에이전트를 도구로 사용**할 수 있게 해줍니다.  
즉, **핸드오프 없이도** 에이전트가 다른 에이전트를 호출할 수 있습니다.

-------------------------

`Runner.run()`을 호출하면, 최종 결과가 나올 때까지 루프가 실행됩니다.  이 루프의 동작 방식은 다음과 같습니다:

1. **LLM 호출:**  
   에이전트에 설정된 모델과 옵션, 그리고 대화 기록(message history)을 바탕으로 LLM을 호출.  

2. **LLM 응답 처리:**  
   모델이 응답을 반환하며, 여기에는 **툴 호출(tool calls)**이 포함될 수 있습니다.

3. **최종 출력이 있는 경우:**  
   응답에 **최종 출력(final output)**이 포함되어 있다면, 이를 반환하고 루프를 종료.

4. **핸드오프가 있는 경우:**  
   응답에 **다른 에이전트로의 핸드오프(handoff)**가 있으면, 현재 에이전트를 새로운 에이전트로 설정한 후 1단계부터 다시 시작.

5. **툴 호출 처리:**  
   툴 호출이 있을 경우, 해당 툴을 실행하고 결과 메시지를 기록한 다음 1단계로 되돌아갑니다.


**Final Output(최종 출력)** 은 루프에서 에이전트가 생성한 마지막 결과물입니다.

- 에이전트에 `output_type`이 설정되어 있다면:  
  → LLM이 해당 형식에 맞는 structured output(구조화된 출력)을 반환할 때 루프가 종료

- `output_type`이 설정되지 않은 경우:  
  → 툴 호출이나 핸드오프가 포함되지 않은 첫 번째 LLM 응답이 최종 출력으로 간주

In [ ]:
# 함수 호출 example
def multiply(x: float, y: float) -> float:
def get_weather(위도: float, 경도: float) -> str:

## Guardrails (가드레일)

**가드레일(Guardrails)** 은 에이전트와 **병렬로 실행되며**, 사용자 입력에 대해 **검사와 유효성 검증**을 수행할 수 있게 해줍니다.

예를 들어, 아주 똑똑하지만 **느리고 비용이 많이 드는 모델**을 사용하는 에이전트를 악의적인 사용자가 수학 숙제를 대신 풀어달라고 요청하는 경우,
빠르고 저렴한 모델을 활용한 가드레일을 실행하여, 사용자의 입력이 악의적인 목적(예: 숙제 대행)인지 먼저 검사할 수 있습니다.  
가드레일이 의심스러운 입력을 감지하면 즉시 오류를 발생시켜 고비용 모델의 실행을 막고 시간과 비용을 절약할 수 있습니다.

In [ ]:
# guardrail 에이전트의 출력 형식을 정의
# is_homework: 숙제인지 여부
# reasoning: 숙제로 판단한 이유 설명
class HomeworkOutput(BaseModel):

In [ ]:
# 숙제 여부를 판단하는 guardrail 전용 에이전트 정의
# 수학 질문 전용 튜터 에이전트 정의
# 역사 질문 전용 튜터 에이전트 정의
# 입력이 '숙제 질문'인지 판단하는 guardrail 함수 정의
# triage_agent에서 InputGuardrail로 사용됨
    # guardrail_agent를 실행하여 숙제 여부 판단
    # 결과를 HomeworkOutput 모델 형태로 변환
    # GuardrailFunctionOutput을 반환
    # is_homework가 False이면 tripwire_triggered를 True로 설정하여 실행을 중단시킴

In [ ]:
# 사용자 질문이 숙제인지 확인하고,
# 숙제라면 수학 튜터 또는 역사 튜터 에이전트로 전달(handoff)하는 판단 에이전트 정의
# triage_agent를 테스트하는 비동기 함수 정의
        # 첫 번째 테스트: 역사 관련 숙제 질문
# main 함수 실행 (Notebook 또는 async 환경에서 사용 가능)

### 구조화된 출력

In [ ]:
# 캘린더 이벤트 데이터 구조 정의
class CalendarEvent(BaseModel):
# 에이전트 정의: 텍스트에서 캘린더 이벤트 정보를 추출
# 실행 함수 정의
# 비동기 함수 실행

### 실습 문제

1. **분류 에이전트**:  
   사용자 입력이 \*\*‘수학 문제’\*\*인지 \*\*‘기타 질문’\*\*인지 분류하세요.  
   분류 결과에 따라 적절한 에이전트로 handoff 하도록 하세요.

3. **수학 에이전트**:  
   수학 문제일 경우, Python 함수 `calculate_area(length: float, width: float)`를 도구로 등록하여 `직사각형의 넓이`를 계산해주는 역할을 수행하세요.

4. **일반 에이전트**:  
   기타 질문에 대해서는 "질문을 이해했지만 수학 관련 질문만 도와드릴 수 있어요." 라고 응답하세요.

5. **가드레일**:  
   입력 내용에 \*\*금지어(`해킹`, `폭탄`)\*\*가 포함되어 있으면 guardrail을 작동시켜 에이전트 실행을 **중단**시키세요.  
   (힌트: InputGuardrail 사용)


### 테스트 입력 예시

* `"가로 5, 세로 7인 직사각형의 넓이를 구해주세요."`  
  👉 수학 에이전트 → 함수 실행 → 넓이 출력  

* `"어제 뉴스에 나온 해킹 사고에 대해 말해줘"`  
  👉 가드레일 트리거 → 실행 중단  

* `"오늘 날씨 어때?"`  
  👉 일반 에이전트 응답